In [4]:
## load modules and run mlflow_logging.ipynb to get function to track model information on MLFLow
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import LinearSVR, SVR
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler


from sklearn.model_selection import GridSearchCV

from modeling.functions import modelling, get_features 


In [5]:
data = pd.read_csv('../data/GEFCom2014Data/Wind/raw_data_incl_features.csv', parse_dates=['TIMESTAMP'])
data.head()
data.dropna(inplace=True)
data.info()
RSEED = 42

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175265 entries, 0 to 175433
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ZONEID      175265 non-null  int64         
 1   TIMESTAMP   175265 non-null  datetime64[ns]
 2   TARGETVAR   175265 non-null  float64       
 3   U10         175265 non-null  float64       
 4   V10         175265 non-null  float64       
 5   U100        175265 non-null  float64       
 6   V100        175265 non-null  float64       
 7   HOUR        175265 non-null  int64         
 8   MONTH       175265 non-null  int64         
 9   WEEKDAY     175265 non-null  int64         
 10  IS_HOLIDAY  175265 non-null  int64         
 11  WS10        175265 non-null  float64       
 12  WS100       175265 non-null  float64       
 13  WD10        175265 non-null  float64       
 14  WD100       175265 non-null  float64       
 15  WD100CARD   175265 non-null  object        
 16  WD

In [6]:
data = pd.get_dummies(data, columns = ['WD100CARD','WD10CARD'])
data.head()

,ZONEID,TIMESTAMP,TARGETVAR,U10,V10,U100,V100,HOUR,MONTH,WEEKDAY,...,WD10CARD_NNW,WD10CARD_NW,WD10CARD_S,WD10CARD_SE,WD10CARD_SSE,WD10CARD_SSW,WD10CARD_SW,WD10CARD_W,WD10CARD_WNW,WD10CARD_WSW
0,1,2012-01-01 01:00:00,0.000000,2.124600,-2.681966,2.864280,-3.666076,1,1,6,...,0,1,0,0,0,0,0,0,0,0
1,1,2012-01-01 02:00:00,0.054879,2.521695,-1.796960,3.344859,-2.464761,2,1,6,...,0,1,0,0,0,0,0,0,0,0
2,1,2012-01-01 03:00:00,0.110234,2.672210,-0.822516,3.508448,-1.214093,3,1,6,...,0,0,0,0,0,0,0,0,1,0
3,1,2012-01-01 04:00:00,0.165116,2.457504,-0.143642,3.215233,-0.355546,4,1,6,...,0,0,0,0,0,0,0,1,0,0
4,1,2012-01-01 05:00:00,0.156940,2.245898,0.389576,2.957678,0.332701,5,1,6,...,0,0,0,0,0,0,0,1,0,0


In [7]:
## train-test-split
data_train, data_test = train_test_split(data, test_size=0.25, random_state=RSEED, stratify=data.ZONEID)
# define features and feature dict
feature_dict = get_features(data)

features = feature_dict['all']

In [9]:
# define zone
zone = 1
# split train and test data in feature and TARGETVAR parts and cut data to desired zones
X_train = data_train[data_train.ZONEID == zone][features]
y_train = data_train[data_train.ZONEID == zone].TARGETVAR

X_test = data_test[data_test.ZONEID == zone][features]
y_test = data_test[data_test.ZONEID == zone].TARGETVAR

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
model = SVR()
for ii in range(5, 6):
    model = SVR(kernel='poly', degree=ii, C=1, cache_size=100)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_train = [1 if value >= 1 else 0 if value <= 0 else value for value in y_pred_train]
    y_pred = model.predict(X_test)
    y_pred = [1 if value >= 1 else 0 if value <= 0 else value for value in y_pred]
    print(ii,': train',mean_squared_error(y_train, y_pred_train, squared=False),
            ', test:', mean_squared_error(y_test, y_pred, squared=False))

5 : train 0.15264726030869225 , test: 0.16490815811588078


In [15]:
data.WD100

0         321.999735
1         306.385781
2         289.088098
3         276.310236
4         263.581938
             ...    
175429     39.369070
175430     29.647245
175431     19.667057
175432     12.686854
175433      5.611802
Name: WD100, Length: 175265, dtype: float64

In [43]:
class WindDirection(int):
    def __new__(cls, value, *args, **kwargs):
        # if value < 0:
        #     raise ValueError("WindDirection types must not be less than zero")
        value = (value+360)%360
        return  super(cls, cls).__new__(cls, value)

    def __add__(self, other):
        res = super(WindDirection, self).__add__(other)
        res = (res+360)%360
        return self.__class__(max(res, 0))

    def __sub__(self, other):
        res = super(WindDirection, self).__sub__(other)
        res = (res+360)%360
        return self.__class__(max(res, 0))

    def __mul__(self, other):
        res = super(WindDirection, self).__mul__(other)
        res = (res+360)%360
        return self.__class__(max(res, 0))

    def __div__(self, other):
        res = super(WindDirection, self).__div__(other)
        res = (res+360)%360
        return self.__class__(max(res, 0))

    def __str__(self):
        return "%d" % int(self)

    def __repr__(self):
        return str(int(self))
        # return "WindDirection(%d)" % int(self)

In [68]:
class WD_NP(np.int64):
    def __new__(cls, value, *args, **kwargs):
        return  super(cls, cls).__new__(cls, value)

    def __add__(self, other):
        res = super(WindDirection, self).__add__(other)
        res = (res+360)%360
        return self.__class__(res)

In [69]:
a = WD_NP(3)
a

3

In [70]:
type(a)

__main__.WD_NP

In [71]:
a = WindDirection(130)
b = WindDirection(3)

In [72]:
a*3

30

In [73]:
data['WindDir'] = data.WD100.apply(lambda x : WD_NP(x))

In [74]:
type(data['WindDir'].iloc[1])

numpy.int64

In [75]:
data['WindDir'].apply(lambda x : 3*x)

0         963
1         918
2         867
3         828
4         789
         ... 
175429    117
175430     87
175431     57
175432     36
175433     15
Name: WindDir, Length: 175265, dtype: int64